## Pipe


*   taking mp3 from folder and  chopping them to 30s piece
*   each of those pieces are transcribed with whisper and output saved to output.txt file 
*  Txt file is summarised with google Pegasus


## Mount drive

In [1]:
#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install ffmpeg-python


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import ffmpeg
import os
import subprocess
from pathlib import Path 

In [4]:

def chop_mp3_files(directory):
    # Create the output directory if it doesn't exist
    output_dir = os.path.join(directory, 'chopped')
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Find all MP3 files in the input directory
    mp3_files = [f for f in os.listdir(directory) if f.endswith('.mp3')]
    
    # Loop through each MP3 file and chop it into 30-second pieces
    for mp3_file in mp3_files:
        input_path = os.path.join(directory, mp3_file)
        output_prefix = os.path.join(output_dir, os.path.splitext(mp3_file)[0])
        subprocess.run(['ffmpeg', '-i', input_path, '-f', 'segment', '-segment_time', '30', '-c', 'copy', '-reset_timestamps', '1', '-map', '0', f'{output_prefix}_%03d.mp3'], check=True)


## 1 Chopping

In [5]:

#@title # Chopping mp3 to 30s pieces with form usage to subdirectory "/chopped" { display-mode: "form" }
folder_with_mp3_to_process = '/content/drive/MyDrive/Whisper/audio/breath' #@param [""] {allow-input: true}
# scale =  800#@param {type:"integer"}
# framerate = 10 #@param {type:"integer"}
# t = 270 #@param {type:"integer"}
chop_mp3_files(folder_with_mp3_to_process)

## 2 Transcribing with whisper 

In [6]:
! pip install git+https://github.com/openai/whisper.git -q
# ! pip install gradio -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.9 MB/s eta 0:00:00


In [7]:
import shutil
import whisper
# import gradio as gr 
import time

model = whisper.load_model("base")


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 162MiB/s]


In [8]:
model.device

device(type='cuda', index=0)

In [11]:
# 
def transcribe(audio):
    
    #time.sleep(3)
    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")

    # decode the audio
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)

    # # Copy the log.csv file to the output folder

    # shutil.copy("/content/flagged/log.csv", "/content/drive/MyDrive/Whisper/output/log.csv")
    return result.text+ '\n'



In [9]:
def batch_trans(directory_path):
    # Get a list of all mp3 files in the directory
    mp3_files = [file for file in os.listdir(directory_path) if file.endswith(".mp3")]
    
    # Sort the list of mp3 files by their names
    mp3_files.sort()
    
    # Loop through each mp3 file and transcribe it
    output_text = ""
    for mp3_file in mp3_files:
        mp3_file_path = os.path.join(directory_path, mp3_file)
        transcription = transcribe(mp3_file_path)
        output_text += transcription
        print(mp3_file)
    
    # Write the output text to a file in the same directory
    with open(directory_path+"/output.txt", "a") as f:
        f.write(output_text)
        f.close()

### Transcribing

In [12]:
#@title ### Transcribing 30s pieces  from subdirectory into text saving -->/output.txt" { display-mode: "form" }
folder_with_mp3_to_process = '/content/drive/MyDrive/Whisper/audio/breath/chopped' #@param [""] {allow-input: true}
# scale =  800#@param {type:"integer"}
# framerate = 10 #@param {type:"integer"}
# t = 270 #@param {type:"integer"}
batch_trans(folder_with_mp3_to_process)

Detected language: en
How_to_Breathe_000.mp3
Detected language: en
How_to_Breathe_001.mp3
Detected language: en
How_to_Breathe_002.mp3
Detected language: en
How_to_Breathe_003.mp3
Detected language: en
How_to_Breathe_004.mp3
Detected language: en
How_to_Breathe_005.mp3
Detected language: en
How_to_Breathe_006.mp3
Detected language: en
How_to_Breathe_007.mp3
Detected language: en
How_to_Breathe_008.mp3
Detected language: en
How_to_Breathe_009.mp3
Detected language: en
How_to_Breathe_010.mp3
Detected language: en
How_to_Breathe_011.mp3
Detected language: en
How_to_Breathe_012.mp3
Detected language: en
How_to_Breathe_013.mp3
Detected language: en
How_to_Breathe_014.mp3
Detected language: en
How_to_Breathe_015.mp3
Detected language: en
How_to_Breathe_016.mp3
Detected language: en
How_to_Breathe_017.mp3
Detected language: en
How_to_Breathe_018.mp3
Detected language: en
How_to_Breathe_019.mp3
Detected language: en
How_to_Breathe_020.mp3
Detected language: en
How_to_Breathe_021.mp3
Detected l

# 3. Summarising with Pegasus

In [13]:
!pip install transformers
!pip install transformers-longformer
!pip install --upgrade transformers
!pip install sentencepiece

!pip install torch

!pip install nltk
!pip install networkx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement transformers-longformer (from versions: none)
ERROR: No matching distribution found for transformers-longformer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in 

In [14]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from tqdm import tqdm
import transformers
print(transformers.__version__)

4.29.2


# summarizing with Pegasus

In [15]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from tqdm import tqdm
import torch

def generate_summary_pegasus(file_path, max_chunk_size, summary_len, num_beams):
    # Check if GPU is available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load the Pegasus model and tokenizer
    model_name = 'google/pegasus-xsum'
    pegasus = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
    tokenizer = PegasusTokenizer.from_pretrained(model_name)

    # Read the contents of the file
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Split the text into chunks
    chunks = [text[i:i+max_chunk_size] for i in range(0, len(text), max_chunk_size)]

    # Generate summaries for each chunk
    summaries = []
    with tqdm(total=len(chunks), desc="Generating Summaries") as pbar:
        for i, chunk in enumerate(chunks):
            # Tokenize the chunk
            input_ids = tokenizer.encode(chunk, truncation=True, max_length=1024, return_tensors='pt').to(device)

            # Generate the summary
            summary_ids = pegasus.generate(input_ids, num_beams=num_beams, max_length=summary_len, early_stopping=True) #was 4 nr of beams
            summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

            # Add summary marker
            marker = f"\n {i+1}s: "
            summaries.append(marker + summary)
            
            pbar.update(1)

    # Combine the summaries into a single output
    output_summary = ' '.join(summaries)

    return output_summary


In [16]:
def save_text_to_file(text, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(text)

In [17]:
#@title ### Summarising text file { display-mode: "form" }
txt_file_to_process = '/content/drive/MyDrive/Whisper/audio/breath/chopped/output.txt' #@param [""] {allow-input: true}

# input_text=  read_file(txt_file_to_process)

max_chunk_size =  512#@param {type:"integer"}
summary_len = 150 #@param {type:"integer"}
num_beams = 8 #@param {type:"integer"}
txt_file_to_save = '/content/drive/MyDrive/Whisper/audio/breath/chopped/pegasus-output.txt' #@param [""] {allow-input: true}
# Generate the summary
summary = generate_summary_pegasus(txt_file_to_process,max_chunk_size, summary_len, num_beams)
save_text_to_file(summary, txt_file_to_save)
# Print the summary
print(summary)

Generating Summaries: 100%|██████████| 267/267 [03:53<00:00,  1.15it/s]


 1s: In this week's episode of the Huberman Lab, we are looking at breathing. 
 2s: In our series of letters from African journalists, film-maker and columnist Farai Sevenzo looks at the importance of breathing. 
 3s: Today we are going to talk about breathing. 
 4s: How do you stop hiccuping? 
 5s: In today's lecture, I'm going to show you how to breathe better. 
 6s: In our series of letters from African journalists, film-maker and columnist Ahmed Rashid looks at the importance of breathing. 
 7s: In this week's episode of The One Show, we're going to look at how we control our breathing. 
 8s: If you want to control your breathing, you have to control your brain. 
 9s: In our series of letters from African journalists, film-maker and columnist Farai Sevenzo looks at the importance of breathing. 
 10s: In this series, I'm going to show you how to breathe better. 
 11s: In this podcast, I'm going to be looking at breathing. 
 12s: This podcast is not intended to be a substitute for p